In [187]:
import numpy as np 
import math
from decimal import *
getcontext().prec = 10

matriz_posicao = [[0.280, 0, 0.300]]
matriz_forca_max  = [[0, 16000, 12000]]
matriz_forca_min = [[0, 0, 0]]
matriz_momento = ''
d= Decimal(0.078)
tensao_ruptura = 1000e6
lim_esc = Decimal(800e6)
aco = 'laminado_quente'
carga = 'axial'
conf = 50
q = 1
kt = 1
temp = 80


# 1 - Sorderberg
# 2 - Goodman
# 3 - Gerber
# 4 - ASME
# 5 - Escoamento Ampliado
criterio = 5

In [188]:
def prod_vet(matriz_posicao, matriz_forca, matriz_momento):
  i = 0
  a = []

  while i < len(matriz_posicao):
      prod = np.cross(matriz_posicao[i], matriz_forca[i])
      prod = a.append(prod)
      prod = sum(a)
      i = i + 1

  if matriz_momento == '':
    matriz_forca = np.sum(matriz_forca, axis=0)
    t = prod[2]
    m = math.sqrt(((prod[0])**2) + ((prod[1])**2))
    f = matriz_forca[2]
  else:
    matriz_forca = np.sum(matriz_forca, axis=0)
    matriz_momento = np.sum(matriz_momento, axis=0)
    prod = prod + matriz_momento
    t = prod[2]
    m = math.sqrt(((prod[0])**2) + ((prod[1])**2))
    f = matriz_forca[2]
  
  print(f'F = {matriz_forca[0]:.2f} i + {matriz_forca[1]:.2f} j + {matriz_forca[2]:.2f} k')
  print(f'M = {prod[0]:.2f} i + {prod[1]:.2f} j + {prod[2]:.2f} k')
    

 
  return m,t,f

In [189]:
#momento máximo e torque máximo
x = prod_vet(matriz_posicao, matriz_forca_max, matriz_momento)

f_max = (x[2])
t_max = Decimal(x[1])
m_max = Decimal(x[0])


#momento mínimo e torque mínimo
y = prod_vet(matriz_posicao, matriz_forca_min, matriz_momento)

f_min = (y[2])
t_min = Decimal(y[1])
m_min = Decimal(y[0])

F = 0.00 i + 16000.00 j + 12000.00 k
M = -4800.00 i + -3360.00 j + 4480.00 k
F = 0.00 i + 0.00 j + 0.00 k
M = 0.00 i + 0.00 j + 0.00 k


In [190]:
# determinar ka
def ka(aco):
  if aco == 'forjado':
    a = 272
    b = -0.995
    ka = a * ((tensao_ruptura* 10**-6) ** b)
  elif aco == 'laminado_quente':
    a = 57.7
    b = -0.718
    ka = a * ((tensao_ruptura* 10**-6) ** b)
  elif aco == 'usinado_ou_laminado_frio':
    a = 4.51
    b = -0.265
    ka = a * ((tensao_ruptura* 10**-6) ** b)
  elif aco == 'retificado':
    a = 1.58
    b = -0.085
    ka = a * ((tensao_ruptura* 10**-6) ** b)
  elif aco == 'espelhado':
    a = 1
    b = 0
    ka = a * ((tensao_ruptura* 10**-6) ** b)
  return ka

if ka(aco) > 1:
  ka = 1
else:
  ka = ka(aco)

In [191]:
#determinar kb
if 2.79 <=  d*10**3 <= 51:
    kb = ((d* 10**3) / Decimal(7.62))**Decimal(-0.107)
elif 51 < d*10**3 <= 254:
  kb = Decimal(1.51) * ((d* 10**3)**Decimal(-0.157))

In [192]:
# determinar kc
def kc(conf):
  if conf == 50:
    kc = 1
    
  elif conf == 90:
      kc = 0.897

  elif conf == 95:
      kc = 0.868

  elif conf == 98:
      kc = 0.836

  elif conf == 99:
      kc = 0.814

  elif conf == 99.99:
      kc = 0.702

  elif conf == 99.9999:
      kc = 0.620

  elif conf == 99.99999:
      kc = 0.584

  return kc

kc = kc(conf)

In [193]:
# determinar kd
def kd(temp):
  kd = 0.9877 + (0.6507 * (10**-3) * temp) - (0.3414 * (10**-5) * (temp**2)) + (0.5621 * (10**-8) * (temp**3)) - (6.246 * (10**-12) * (temp**4))

  return kd

if kd(temp) > 1:
  kd = 1
else:
  kd = kd(temp)

In [194]:
# determinar kf
kf = 1

In [195]:
#limites de resistência
lim_normal_res = Decimal(ka)* Decimal(kb) * Decimal(kc) * Decimal(kd) * Decimal(1) * Decimal(kf) * Decimal(0.5) * Decimal(tensao_ruptura)
lim_cisalhante_res = Decimal(ka)* Decimal(kb) * Decimal(kc) * Decimal(kd) * Decimal(0.577) * Decimal(kf) * Decimal(0.5) * Decimal(tensao_ruptura)

In [196]:
tensao_cisalhante_esc = Decimal(0.577) * lim_esc

In [197]:
# determinar o kp
def kp(q,kt):
    kp = 1 + (q * (kt - 1))
    return kp  

In [198]:
kp = Decimal(kp(q,kt))

In [199]:
pi = Decimal(math.pi)

In [200]:
tensao_cisalhante_esc = Decimal(0.577) * lim_esc

tensao_normal_max = (kp * ((Decimal(32) * m_max) / (Decimal(pi) * (d**Decimal(3))))) + (kp * ((Decimal(4) * f_max) / (Decimal(pi) * (d**Decimal(2)))))
tensao_normal_min = (kp * ((Decimal(32) * m_min) / (Decimal(pi) * (d**Decimal(3))))) + (kp * ((Decimal(4) * f_min) / (Decimal(pi) * (d**Decimal(2)))))
tensao_cisalhante_max = (kp * ((Decimal(16) * t_max) / (Decimal(pi) * (d**Decimal(3)))))
tensao_cisalhante_min = (kp * ((Decimal(16) * t_min) / (Decimal(pi) * (d**Decimal(3)))))

tensao_normal_var = (tensao_normal_max - tensao_normal_min) / Decimal(2)
tensao_normal_media = (tensao_normal_max + tensao_normal_min) / Decimal(2)

tensao_cisalhante_media = (tensao_cisalhante_max + tensao_cisalhante_min) / Decimal(2)
tensao_cisalhante_var = (tensao_cisalhante_max - tensao_cisalhante_min) / Decimal(2)
  

In [201]:
if criterio == 1:
  #sorderberg 
  def sorderberg(tensao_normal_media,tensao_normal_var,lim_esc,lim_normal_res,tensao_cisalhante_media,tensao_cisalhante_var,tensao_cisalhante_esc,lim_cisalhante_res):
    tensao_normal_sorderberg = tensao_normal_media + (tensao_normal_var * (lim_esc / lim_normal_res))
    tensao_cisalhante_sorderberg = tensao_cisalhante_media + (tensao_cisalhante_var * (tensao_cisalhante_esc / lim_cisalhante_res))
    fat_seg_sorderberg = lim_esc / ((tensao_normal_sorderberg**Decimal(2)) + (Decimal(3) * (tensao_cisalhante_sorderberg**Decimal(2))))**(Decimal(1/2))

    return print(f'Tensão normal sorderberg: {tensao_normal_sorderberg} \nTensão cisalhante sorderberg: {tensao_cisalhante_sorderberg} \nN sorderberg: {fat_seg_sorderberg}')

  sorderberg(tensao_normal_media,tensao_normal_var,lim_esc,lim_normal_res,tensao_cisalhante_media,tensao_cisalhante_var,tensao_cisalhante_esc,lim_cisalhante_res)

elif criterio == 2:
  #modificado de goodman
  def mod_good(lim_esc,tensao_normal_var,lim_normal_res,tensao_normal_media,tensao_ruptura,tensao_cisalhante_esc,tensao_cisalhante_var,lim_cisalhante_res,tensao_cisalhante_media):
    tensao_normal_mod_god = lim_esc * ((tensao_normal_var/lim_normal_res) + (tensao_normal_media/Decimal(tensao_ruptura)))
    tensao_cisalhante_mod_god = tensao_cisalhante_esc * ((tensao_cisalhante_var/lim_cisalhante_res) + (tensao_cisalhante_media/(Decimal(tensao_ruptura)*Decimal(0.8))))
    fat_seg_mod_god = lim_esc / (((tensao_normal_mod_god**Decimal(2)) + Decimal(3) * (tensao_cisalhante_mod_god**Decimal(2)))**(Decimal(1/2)))

    return print(f'Tensão normal modificado de goodman: {tensao_normal_mod_god} \nTensão cisalhante modificado de goodman: {tensao_cisalhante_mod_god} \nN modificado de goodman: {fat_seg_mod_god}')

  mod_good(lim_esc,tensao_normal_var,lim_normal_res,tensao_normal_media,tensao_ruptura,tensao_cisalhante_esc,tensao_cisalhante_var,lim_cisalhante_res,tensao_cisalhante_media)

elif criterio == 3:
  #gerber
  def gerber(lim_esc,tensao_normal_var,lim_normal_res,tensao_normal_media,tensao_ruptura):
    a1 = Decimal(1)
    b1 = (lim_esc * tensao_normal_var) / lim_normal_res
    c1 = ((lim_esc * tensao_normal_media) / Decimal(tensao_ruptura))**Decimal(2)

    delta1 = b1*b1 - (Decimal(4)*a1*(-c1))

    tensao_normal_gerber2 = (-b1 - Decimal(math.sqrt(delta1)) ) / (Decimal(2)*a1)
    print(f'Tensão normal de gerber: {tensao_normal_gerber2}' )

    a2 = Decimal(1)
    b2 = (tensao_cisalhante_esc * tensao_cisalhante_var) / lim_cisalhante_res
    c2 = ((tensao_cisalhante_esc * tensao_cisalhante_media) / (Decimal(tensao_ruptura) * Decimal(0.8)))**Decimal(2) 

    delta2 = b2*b2 - (Decimal(4)*a2*(-c2))
    
    tensao_cisalhante_gerber2 = (-b2 - Decimal(math.sqrt(delta2)) ) / (Decimal(2)*a2)
    print(f'Tensão cisalhante de gerber: {tensao_cisalhante_gerber2}' )

    fat_seg_gerber = lim_esc / (((tensao_normal_gerber2 **Decimal(2)) + Decimal(3) * (tensao_cisalhante_gerber2 **Decimal(2)))**(Decimal(1/2)))
    return print(f'N gerber: {fat_seg_gerber}')

  gerber(lim_esc,tensao_normal_var,lim_normal_res,tensao_normal_media,tensao_ruptura)

elif criterio == 4:
  #asme
  def asme(lim_esc,tensao_normal_var,lim_normal_res,tensao_normal_media,tensao_cisalhante_esc,tensao_cisalhante_var,lim_cisalhante_res,tensao_cisalhante_media):
    tensao_normal_asme = Decimal(math.sqrt((((lim_esc * tensao_normal_var)/lim_normal_res)**Decimal(2)) + ((tensao_normal_media)**Decimal(2))))
    tensao_cisalhante_asme = Decimal(math.sqrt((((tensao_cisalhante_esc * tensao_cisalhante_var)/lim_cisalhante_res)**Decimal(2)) + ((tensao_cisalhante_media)**Decimal(2))))
    fat_seg_asme = lim_esc/Decimal(math.sqrt((tensao_normal_asme**Decimal(2)) + Decimal(3)*(tensao_cisalhante_asme**Decimal(2))))

    print(f'Tensão normal ASME: {tensao_normal_asme}')
    print(f'Tensão cisalhante ASME: {tensao_cisalhante_asme}')
    print(f'N ASME: {fat_seg_asme}')
    return 

  asme(lim_esc,tensao_normal_var,lim_normal_res,tensao_normal_media,tensao_cisalhante_esc,tensao_cisalhante_var,lim_cisalhante_res,tensao_cisalhante_media)

elif criterio == 5:
  #escoamento ampliado
  def esc_amp(tensao_normal_var,tensao_normal_media,tensao_cisalhante_var,tensao_cisalhante_media,lim_esc):
    tensao_normal_escamp = tensao_normal_var + tensao_normal_media
    tensao_cisalhante_escamp = tensao_cisalhante_var + tensao_cisalhante_media
    fat_seg_escamp = lim_esc/((tensao_normal_escamp**Decimal(2)) + Decimal(3)*(tensao_cisalhante_escamp**Decimal(2)))**(Decimal(1/2))

    print(f'Tensão normal escoamento ampliado: {tensao_normal_escamp}')
    print(f'Tensão cisalhante escoamento ampliado: {tensao_cisalhante_escamp}')
    print(f'N escoamento ampliado {fat_seg_escamp}')

    return
  
  esc_amp(tensao_normal_var,tensao_normal_media,tensao_cisalhante_var,tensao_cisalhante_media,lim_esc)

Tensão normal escoamento ampliado: 128273677.9
Tensão cisalhante escoamento ampliado: 48079984.16
N escoamento ampliado 5.230971890
